In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sequential.preprocessing import preprocessing
from sequential.metrics import mean_absolute_error
from sequential.models import Transformer

### Load data

In [ ]:
df = pd.read_csv('../data/AirPassengers.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.columns

### Preprocessing

In [ ]:
val_len = 12
time_steps = 120
target_time_steps = 12

X = df.to_numpy()
X_orig = X

X, X_train, X_test, y_train, y_test, scaler = preprocessing(X, val_len, time_steps, target_time_steps, autoregressive=True)

y = scaler.transform(X_orig)[:, 0].flatten()

# print the training data shape: (num_sequences, time_steps, num_features)
X_train.shape, y_train.shape

### Model fit

In [ ]:
%%time

# set a random seed for repeatable results
# np.random.seed(7)

optimizer_args = {'alpha': .001}

model = Transformer(num_decoder_layers=6, d_model=50, units=64, num_heads=1, 
                    optimizer_args=optimizer_args, normalize=False, loss='mae', 
                    drop_rate=0, apply_positional_encoding=False)

fitted_val, loss = model.fit(X_train, y_train, epochs=1000, verbose_rate=100)

### Fitted values plot

In [ ]:
plt.plot(y_train.flatten())
plt.plot(fitted_val.flatten())
print('Fit MAE: ', mean_absolute_error(y_train.flatten(), fitted_val.flatten()))

### Validation forecast

In [ ]:
forecast = model.get_forecast(target_time_steps, X_test).flatten()
forecast

In [ ]:
plt.plot(y)
forecast_len = len(forecast)
# create an empty array with the forecast values at the end
forecast_array = np.zeros(len(y))
forecast_array[:] = np.inf
forecast_array[-forecast_len:] = forecast
# forecast_array = np.append(forecast_array, forecast)
# forecast_array[-test_set_length:-test_set_length + forecast_len] = forecast
plt.plot(forecast_array)

print('MAE:', mean_absolute_error(y_test, forecast[:val_len]))

In [ ]:
plt.plot(y_test)
plt.plot(forecast[:val_len])
print('MAE:', mean_absolute_error(y_test, forecast[:val_len]))

### Forecast beyond historical data

In [ ]:
forecast = model.get_forecast(target_time_steps, X[-1][np.newaxis, :, :]).flatten()

In [ ]:
plt.plot(y)
# create an empty array with the forecast values at the end
forecast_array = np.zeros(y.size + forecast.size)
forecast_array[:] = np.inf
forecast_array[-forecast.size:] = forecast
plt.plot(forecast_array)